In [ ]:
import glob
import itertools
import pandas as pd

In [ ]:
MAPPING = (
    ('Barschaft (teilweise eigene Berechnung)', 'barschaft'),
    ('Gulden Inventarsumme', 'invenatar_summe_norm_fl'),
    ('Kreuzer Inventarsumme', 'invenatar_summe_norm_kr'),
    ('Gulden vor Abzug Passiva', 'vor_passiva_fl'),
    ('Kreuzer vor Abzug Passiva', 'vor_passiva_kr'),
    ('Gulden nach Abzug Passiva', 'nach_passiva_fl'),
    ('Kreuzer nach Abzug Passiva', 'nach_passiva_kr'),
    ('Buch/Bücher', 'buecher'),    
)

In [ ]:
orig_files = glob.glob('../rita_data_ibk/source/*.xlsx')

In [ ]:
dfs = []
for file in orig_files:
    keys = list(pd.read_excel(file, None).keys())
    for x in keys:
        df = pd.read_excel(file, x)
        dfs.append(df)

In [ ]:
inst, _ = Institution.objects.get_or_create(written_name="Südtiroler Landesarchiv")

In [ ]:
doktype_col, _ = SkosCollection.objects.get_or_create(name='adm-action-type')

In [ ]:
def create_entry(row, repo_name='Südtiroler Landesarchiv'):
    signatur = row['Signatur']
    vb = signatur.split(',')[0]
    year = vb[-4:]
    vb_sig = "{}, {}".format(repo_name, vb)
    vb_obj, _ = VerfachBuch.objects.get_or_create(signatur=vb_sig)
    inv, _ = InventoryEntry.objects.get_or_create(inv_signatur="{}, {}".format(repo_name, signatur))
    inv.is_located_in = vb_obj
    inv.save()
    return inv
    

In [ ]:
def relate_person(row, rel_type, persons, entry):
    for y in persons:
        if y.startswith(')'):
            pers = None
        elif y.startswith('[Vorstehe'):
            pers = None
        elif y.startswith('Im Rahm'):
            pers = None
        elif y.startswith('[die Personen'):
            pers = None
        elif y.startswith('n.a'):
            pers = None
        else:
            try:
                pers, _ = Person.objects.get_or_create(written_name=y)
            except Exception as e:
                pers = None
        if pers:
            getattr(entry, rel_type, None).add(pers)
    return entry
            
       

In [ ]:
def add_more_data(row, entry, mapping):
    for item in mapping:
        setattr(entry, item[1], row[item[0]])
        try:
            entry.save()
        except Exception as e:
            setattr(entry, item[1], None)
    return entry

In [ ]:
for x in dfs:
    for i, row in x.iterrows():
        entry = create_entry(row)
        job_all = row['Beruf']
        if "n. a." in job_all:
            job = None
        elif ":" in job_all:
            job = job_all.split(': ')[-1]
        else:
            job = job_all
        name_main = row['Name\n(Erklärung aus Verfachbuch)'].split('\n')[0].split('; ')
        main_place = row['Wohnort; Ort des betreffenden Gutes/der betreffenden Behausung'].split('\n')[0]
        for name in name_main:
            if job:
                full_name = "{}, {}, {}".format(name, job, main_place)
            else:
                full_name = "{}, {}".format(name, main_place)
            try:
                main_pers, _ = Person.objects.get_or_create(written_name=full_name)
            except Exception as e:
                main_pers = None
            if main_pers:
                entry.main_person.add(main_pers)
        doc_type_str = row['Dokumententypus'].split('\n')[0].split(';')[0]
        doc_type, _ = SkosConcept.objects.get_or_create(pref_label=doc_type_str)
        doc_type.collection.add(doktype_col)
        entry.inv_type = doc_type
        if "[KEIN umfassendes Inventa" in row['Dokumententypus']:
            entry.vollstaendig = 'unvollständig'
        if row['Buch/Bücher'].startswith('j'):
            entry.buecher_sys = 'Bücher'
        entry.excel_row = row.to_json(force_ascii=False)
        persons = str(row['Beteiligte Personen (Beamte, Gerichtsverpflichtete, Zeugen, ...)']).split('\n')
        relate_person(row, 'adm_person', persons, entry)
        persons = str(row['Beteiligte Personen (Erbsinteressenten, Gerhaben, Anweiser, Verkäufer, Verpächter, Käufer, Pächter, ...)']).split('\n')
        relate_person(row, 'related_person', persons, entry)
        persons = str(row['Genannte Personen']).split('\n')
        relate_person(row, 'other_person', persons, entry)
        entry = add_more_data(row, entry, MAPPING)
        entry.save()

In [ ]:
# store number of related persons
# [x.save_stats() for x in InventoryEntry.objects.all()]

In [ ]:
# merge duplicated Verfachbücher
# from django_super_deduper.merge import MergedModelInstance

In [ ]:
# for x in VerfachBuch.objects.all():
#     if "Michae" in x.signatur:
#         loc_filter = "Michae"
#     elif "Oberamtsgerich" in x.signatur:
#         loc_filter = "Oberamtsgerich"
#     else:
#         loc_filter = "Stadtgericht"
#     year = str(x.year)[:4]
#     vfb =VerfachBuch.objects.exclude(id=x.id).filter(
#         signatur__icontains=year
#     ).filter(signatur__icontains='Süd').filter(
#         signatur__icontains=loc_filter
#     )
#     if vfb.count() == 1:
#         merged_object = MergedModelInstance(x, vfb[0]).merge(vfb[0])
#         vfb[0].delete()
#         print(f"merged {x}")

In [ ]:
# Link inventory entries to verfachbuchentries
# hits = 0
# zeros = 0
# more = 0
# more_obj = []
# offset = -5
# for x in InventoryEntry.objects.all():
#     vfb = x.is_located_in
#     sig = x.inv_signatur
#     entrys = VfbEntry.objects.filter(located_in=vfb)
#     matching_str = sig[offset:]
#     matches = VfbEntry.objects.filter(located_in=vfb).filter(entry_signatur__endswith=matching_str)
#     if matches.count() == 0:
#         zeros += 1
#     elif matches.count() == 1:
# #         my_obj = matches[0]
# #         my_obj.inventory_entry = x
# #         my_obj.save()
#         hits += 1
#     else:
#         more_obj.append((x, matches))
#         more += 1
#     if matches.count() > 0:
#         x.vfb_entry.set(matches)
# print(offset, hits, zeros, more)


In [ ]:
# InventoryEntry.objects.all().delete()

In [ ]:
# Person.objects.all().delete()

In [ ]:
# VerfachBuch.objects.all().delete()